In [1]:
import cdsapi
import argparse
from urllib.request import urlopen
import os
import glob
import math
import time
import datetime
import random
random.seed(3019)
from copy import deepcopy
import warnings
import itertools
import scipy.stats
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.cm import ScalarMappable
import multiprocess as mp
import scipy.io
warnings.simplefilter(action='ignore')

In [3]:
print('------------------read data------------------')
print('------------------select t2mx or t850m as the target-------------------')
# If select t2mx, need to use sliding windows to filter out the isolated anomalies; 
# Otherwise using t850m, it is not necessary to filter out the isolated anomalies cuz the t850m field should be much smoother
data2d = xr.open_mfdataset('/burg/glab/users/yh3019/era5_daily/atmos3d/temperature/temperature*.nc').sel(level=850).reset_coords('level',drop=True)
data2d = data2d.rename({'t':'t850'})
data2d2 = xr.open_dataset('/burg/glab/users/yh3019/era5_daily/surface2d/2m_temperature_dailymax.nc') #['t2m']  #.sel(longitude=slice(10,30),latitude=(35,60))
data2d2 = data2d2.rename({'t2m':'t2mx'})
data2d['t2mx'] = data2d2['t2mx']
mask =  xr.open_dataset('/burg/glab/users/yh3019/data2d_dkrz/landmask.nc')

ll1 = np.arange(0,180.25,0.25)
ll2 = np.arange(-179.75,0,0.25)
ll = np.concatenate([ll1,ll2])
mask['longitude'] = ll
mask = mask.sortby('longitude')
mask = mask.sel(latitude=slice(70,15),longitude=slice(-30,40)).isel(time=0)
data2d['mask'] = mask.lsm
data2d = data2d.where(data2d.mask>0.9)

data2d = data2d.sortby('latitude')
# hw = data2d['t850'] # t2mm # t850 # swvl1 # z500m
# hw = data2d['t2mx']

------------------read data------------------
------------------select t2mx or t850m as the target-------------------


In [4]:
# hw = data2d['t850'].to_dataset()
hw = data2d['t2mx'].to_dataset()

MJJA = hw.sel(time=hw.time.dt.month.isin([5, 6, 7, 8])) # ['t850']
MJJA['doy'] = xr.DataArray(data = np.repeat(np.arange(0,123).reshape(1,123), 33, axis=0).reshape(123*33),
             dims=['time'],
             coords=dict(time=MJJA.time))
MJJA_df = MJJA.to_dataframe().reset_index().set_index(['latitude','longitude']).dropna()
iid = MJJA_df.index.unique()
num_start = 1990
num_end = 2023
years = num_end - num_start
len_year = 61 # June and July, length of days in a year
percentile = 95.
percentile_window = 15 
half_window = 7
start_day = 31


In [52]:
#----------- test with 2022 data, the threshold file is new version now: 95th_15d-----------
t850m_95th = pd.read_csv('/burg/glab/users/yh3019/csv/t850m_95th_11d.csv', index_col=None)[['latitude','longitude','doy','t850m_95th']]
t2mx_95th = pd.read_csv('/burg/glab/users/yh3019/csv/t2mx_95th_15d.csv', index_col=None)[['latitude','longitude','doy','t2mx_95th']]

# Should I use doy? considering the impact of # days in Feb
threshold = t850m_95th.set_index(['doy','latitude','longitude']).to_xarray()['t850m_95th'].sel(longitude=slice(-20,40),latitude=slice(15,70),doy=slice(152,212)) # June and July 
threshold = t2mx_95th.set_index(['doy','latitude','longitude']).to_xarray()['t2mx_95th'].sel(longitude=slice(-20,40),latitude=slice(15,70),doy=slice(0,61)) # June and July 

JJ = MJJA.sel(time=MJJA.time.dt.month.isin([6, 7,])) # ['t850']
data = JJ.sel(longitude=slice(-20,40), latitude=slice(15,70))
consecutive_days = 3

threshold = threshold.sel(latitude=slice(35,60),longitude=slice(-15,20))
data = data.sel(latitude=slice(35,60),longitude=slice(-15,20))

T = np.array(data.t2mx) # all year T values
thres = np.array(threshold)

# keep the latitude longitude info for later process
all_lat = np.repeat(threshold.latitude.values, len(threshold.longitude)).reshape([T[0].shape[0], T[0].shape[1]])
all_lon = np.repeat(threshold.longitude.values, len(threshold.latitude)).reshape([T[0].shape[1], T[0].shape[0]]).T


In [54]:
# hyper parameters
hot_pixel_ratio = 0.6
overlap_threshold = 0.4 
s = int(4/0.25) # sliding_window_size = 16
L = 1000 # 1000km based on Rossby wave size
N = int(hot_pixel_ratio*s*s) # hot_pixel_number
Area = 500000 # km2
# M = Area/(25*25) = 800 # minimum hot pixel number each day (naively equivalent) 
M = 500
consecutive_days = 3


In [84]:
from HW_detectracker import HeatwaveTracker
results = {}

for yr in list(range(years)):
    yr = str(yr+1990)
    T = np.array(data.t2mx.sel(time = yr))
    hw_tracker = HeatwaveTracker(yr, all_lat, all_lon, T, thres, s, N, M, L, overlap_threshold, consecutive_days)
    hw = hw_tracker.detect_heatwaves()
    results[yr] = hw[yr]


1990 : No non-isolated hot pixels detected
1991 : HW event dictionary saved successfully to file
1992 : HW event dictionary saved successfully to file
1993 : No non-isolated hot pixels detected
1994 : HW event dictionary saved successfully to file
1995 : HW event dictionary saved successfully to file
1996 : HW event dictionary saved successfully to file
1997 : HW event dictionary saved successfully to file
1998 : HW event dictionary saved successfully to file
1999 : HW event dictionary saved successfully to file
2000 : HW event dictionary saved successfully to file
2001 : HW event dictionary saved successfully to file
2002 : HW event dictionary saved successfully to file
2003 : HW event dictionary saved successfully to file
2004 : HW event dictionary saved successfully to file
2005 : HW event dictionary saved successfully to file
2006 : HW event dictionary saved successfully to file
2007 : HW event dictionary saved successfully to file
2008 : HW event dictionary saved successfully to f

In [89]:
results

{'1990': {},
 '1991': {},
 '1992': {},
 '1993': {},
 '1994': {1: {'start_day': 26,
   'end_day': 28,
   'duration': 3,
   'center_lat': [(49.23656845753899, 15.432842287694974),
    (49.43533772652389, 15.629324546952224),
    (48.98973004694836, 11.395539906103286)],
   'Tmean Tmax': [(304.61243, 308.94553),
    (305.04453, 309.8987),
    (305.82144, 312.49335)]},
  2: {'start_day': 56,
   'end_day': 60,
   'duration': 5,
   'center_lat': [(51.27450980392157, 12.566993464052288),
    (51.65588521400778, 13.393482490272374),
    (51.513698630136986, 16.782106164383563),
    (50.62182320441989, 16.20524861878453),
    (51.03371993127148, 14.660008591065292)],
   'Tmean Tmax': [(305.7093, 307.9652),
    (306.20956, 309.67072),
    (306.82062, 309.95624),
    (307.15662, 311.21774),
    (307.11905, 311.50323)]}},
 '1995': {1: {'start_day': 48,
   'end_day': 51,
   'duration': 4,
   'center_lat': [(40.46948818897638, -4.164370078740157),
    (44.846860986547085, 0.107847533632287),
    (48

In [83]:
class HeatwaveTracker:
    def __init__(self, yr, all_lat, all_lon, T, thres, s, N, M, L, overlap_threshold, consecutive_days):
        """
        Initialize HeatwaveTracker object.
        
        Args:
        - yr: year.
        - T: 3D array of temperature data in the form of (day, lat, lon).
        - thres: 3D array of temperature threshold data in the form of (day, lat, lon).
        - s: sliding window size.
        - N: integer threshold for the number of hot pixels in a sliding window to be considered a heatwave event.
        - M: integer threshold for the number of hot pixels in a minimum coverage heatwave.
        - L: float distance threshold in degrees for two heatwave events on consecutive days to be considered the same event.
        - overlap_threshold: float overlap threshold (in terms of area or indicies) for two heatwave events on consecutive days to be considered the same event.
        - consecutive_days: the minimum lasting duration of one heatwave event.
        """
        # all_lat, all_lon, T, thres, s, N, M, L, overlap_threshold, consecutive_days = glb.hyperparams
        self.T = T
        self.thres = thres
        self.all_lat = all_lat
        self.all_lon = all_lon
        self.s = s
        self.N = N
        self.M = M
        self.L = L
        self.overlap_threshold = overlap_threshold
        self.consecutive_days = consecutive_days
        
        # Initialize internal variables for _process_day
        self.current_hot_indice = {} # dictionary containing current day non-isolate hot pixel indices, key is day
        self.current_center = {} # dictionary containing center location (lat, lon) of current day (day is the key value in self.current_hot_indice)
        self.current_intensity = {} # dictionary containing mean and max T values over the hot pixels of current day (day is the key value in self.current_hot_indice)
        
        # Initialize internal variables for _merge_events 
        self.events = {} # dictionary of heatwave events start, end days and durations
        

        
    def _process_day(self, day):
        """
        Process each single day of temperature data to detect hot pixels and remove isolated/sparse hot anomalies.
        
        Args:
        - day: integer representing the (current) day to process.
        
        Outputs:
        - self.current_hot_indice
        
        """
        # Generate boolean mask of hot pixels for current day
        hot_pixels = self.T[day] > self.thres[day] # np.where(self.T[day] > self.thres[day], 1, 0)
        hot_indicies = np.arange(0, hot_pixels.shape[0]*hot_pixels.shape[1]).reshape([hot_pixels.shape[0],hot_pixels.shape[1]])
        hot_indicies = hot_indicies*hot_pixels.astype(int)
        new_hot_indice = np.array([0])
        
        # Loop over sliding windows
        for i in range(self.T.shape[1]-15):
            for j in range(self.T.shape[2]-15):
                # Check if sliding window contains enough hot pixels to be considered as a non-anomalies event
                if hot_pixels[i:i+s, j:j+s].sum() >= self.N:
                    tmp_new_hot_indice = hot_indicies[i:i+s, j:j+s].reshape(-1)
                    new_hot_indice = np.unique(np.append(new_hot_indice, tmp_new_hot_indice))            
            
        new_hot_indice = new_hot_indice[new_hot_indice != 0]

        # Set a minimum total hot pixel coverage area - about the area of 1/2 France
        if (len(new_hot_indice) > self.M):
            #calculate the center location, mean and max T values
            clat = np.mean(all_lat[np.isin(hot_indicies, new_hot_indice)])
            clon = np.mean(all_lon[np.isin(hot_indicies, new_hot_indice)])
        
            meanT = np.mean(T[day][np.isin(hot_indicies, new_hot_indice)])
            maxT = np.max(T[day][np.isin(hot_indicies, new_hot_indice)])
            
            self.current_center[day] = (clat, clon)
            self.current_intensity[day] = meanT, maxT
            self.current_hot_indice[day] = new_hot_indice 

    
    
    def _satis_distance_or_overlap_criteria(self, end_day, day):
        # For day and day+1, calculate the center distance
        R = 6371.0  # Approximate radius of earth in km
        lat1, lon1  = self.current_center[end_day]
        lat2, lon2 = self.current_center[day]
        lat1 = np.radians(lat1)
        lat2 = np.radians(lat2)
        lon1 = np.radians(lon1)
        lon2 = np.radians(lon2)
        dlon = np.abs(lon1 - lon2)
        dlat = np.abs(lat1 - lat2)

        a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
        c = 2 * math.atan2(np.sqrt(a), np.sqrt(1 - a))
        distance = R * c
                  
        # Calculate the area overlap ratio
        setA = set(self.current_hot_indice[end_day])
        setB = set(self.current_hot_indice[day])
        overlap = setA & setB
        smaller_set = np.min([len(setA), len(setB)])
        overlap_ratio = float(len(overlap) / smaller_set)
           
        # Meetinge one criteria would be enough
        if (overlap_ratio >= self.overlap_threshold) or (distance <= L):
            return True
        else:
            return False
        
                                  
                    
    def _merge_events(self, ):
        # For one hw event, the starting and ending day
        start_day = 0
        end_day = 0
        duration = 0
        last_day = list(self.current_hot_indice)[-1] # last day with non-isolated hot pixels in this year
        current_event = False # If there is an existing event 
        current_event_id = 0 # ID of current heatwave event
        
        # Only days with hot pixels that satisfies the _process_day function would be looped here
        for day in self.current_hot_indice.keys():
            
            # If currently, the new day is not within an existing hw event current_event = False, then start a hw event
            # Attention: use ~ True instead of ! True
            if (not current_event) and (day < last_day - 1):
                start_day = day
                end_day   = day
                duration = end_day - start_day + 1
                current_event = True
                
            # If currently, the new day is within an existing hw event 
            elif current_event: 
                # If the current day and the end day from the event are consecutive
                if day == end_day + 1:
                    # Further check if the hw coverages in the two days overlap or the centers of hw coverages are within the distance L
                    criteria_check = self._satis_distance_or_overlap_criteria(end_day, day)
                    if criteria_check:
                        # Update the duration and end day info
                        end_day = day
                        duration = end_day - start_day + 1
                        # Finalize last event when reaching end of the days
                        if day == last_day:
                            current_event = False
                    else:
                        # Do not update duration and end day if the criteria is not met, just end the event
                        current_event = False      
                # If the new day and end day are not consecutive, end the hw event
                else:
                    current_event = False               
            
            # For the ended event, check if the hw duration is larger than the minimum required length
            if (duration > 0) and (not current_event):
                if duration >= 3:
                    current_event_id += 1
                    self.events[current_event_id] = {'start_day': start_day, 'end_day': end_day, 'duration': duration} 
                
                # If the event is not ended the last day (namaly, not consecutive or not merged not situations)
                # The current day needs to be counted as the first day of a new event
                start_day = day
                end_day   = day
                duration = end_day - start_day + 1
                current_event = True 
                
                
        
    def detect_heatwaves(self):
        """
        Detect heatwave events and track their movement across days.
        
        Returns:
        - events: dictionary containing all heatwave events
                  with keys as event IDs and values as dictionaries containing 
                  duration, start day, end day.
        """
        if __name__ == '__detect_heatwaves__':
              main()
        
        # Process each day to select hot pixels and remove the sparse isolated anomalies
        for day in range(self.T.shape[0]):
            self._process_day(day)
        
        # Loop all days from last step and merge the consecutive events that meet certain criteria
        if len(self.current_hot_indice) > 0:
            self._merge_events()
        
            # Merge the dictionaries current_hot_indice, current_center, and current_intensity info into events
            for id in self.events.keys():
                self.events[id]['center_lat'] = list( map(self.current_center.get, np.arange(self.events[id]['start_day'], self.events[id]['end_day']+1)) )
                self.events[id]['Tmean Tmax'] = list( map(self.current_intensity.get, np.arange(self.events[id]['start_day'], self.events[id]['end_day']+1)) )
        
            # Load pickle module and save the dictionary events         
            import pickle
            with open('/burg/glab/users/yh3019/csv/hw'+yr+'.pkl', "wb") as fp:
                pickle.dump(self.events, fp)  
                print(yr, ': HW event dictionary saved successfully to file')
        else:
            print(yr, ': No non-isolated hot pixels detected')
            
        # When returning, save the whole dictionary as the value to the year id (key is the str year) 
        yr_events = {yr: self.events}
        return yr_events
    
     

In [321]:
# Show the last year
hw_tracker = HeatwaveTracker(yr, all_lat, all_lon, T, thres, s, N, M, L, overlap_threshold, consecutive_days)
hw = hw_tracker.detect_heatwaves()


dictionary saved successfully to file


In [322]:
hw

{1: {'start_day': 9,
  'end_day': 18,
  'duration': 10,
  'center_lat': [(40.076620825147344, -5.605599214145383),
   (39.98089171974522, -5.212181528662421),
   (40.38673232908459, -3.9020857473928157),
   (39.957040572792366, -4.285202863961814),
   (41.59672386895476, -2.5427067082683306),
   (42.95418448381185, -0.797648136835675),
   (42.56410256410256, -1.0316091954022988),
   (44.80847803881512, -0.025791624106230846),
   (46.878643852978456, 4.329319814110689),
   (49.49252536038441, 10.383208756006407)],
  'Tmean Tmax': [(294.6301, 296.62976),
   (295.572, 299.50964),
   (295.59415, 300.13733),
   (296.6979, 300.35535),
   (296.20065, 300.905),
   (294.7877, 299.49857),
   (295.20078, 299.06308),
   (294.3979, 300.16727),
   (293.9278, 299.36594),
   (292.61078, 296.6994)]},
 2: {'start_day': 41,
  'end_day': 52,
  'duration': 12,
  'center_lat': [(41.10110294117647, -4.4316176470588236),
   (43.70200892857143, -1.5245535714285714),
   (43.08291298865069, -0.05548549810844893)